In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)

# Example: Use Gemini
chain = prompt | gemini_llm
response = chain.invoke({"product": "smart home devices"})
print("Gemini response:", response)

# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)
import langchain
print("LangChain version:", langchain.__version__)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
Gemini response: content='Here are some creative names for a smart home device company, categorized by the vibe they evoke:\n\n**I. Evoking Intelligence & Seamlessness:**\n\n1.  **Aura Automate:** Suggests an invisible, intelligent presence.\n2.  **Synapse Living:** Highlights the interconnected "brain" of the home.\n3.  **EchoLogic:** Implies responsiveness and smart decision-making.\n4.  **OmniSense Home:** Suggests comprehensive awareness and control.\n5.  **CogniHabit:** Combines cognitive intelligence with home habits.\n6.  **Weave Living:** Implies seamlessly integrated technology into the fabric of life.\n7.  **Flux Home Systems:** Suggests dynamic, adaptable intelligence.\n8.  **Nexa Dwell:** "Nexa" for next-gen/nexus, "Dwell" for home.\n9.  **Lumina AI:** "Lumina" for light/clarity, combined with AI.\n10. **Aether Living:** Aether implies a refined, pervasive essence.\n\n**II. Emphasizing Comfort &

# Method 1: Single Agent (Automatic - Recommended)


In [3]:
from langchain.tools import tool
from langchain.agents import create_agent

@tool
def search_customers(query: str) -> str:
    """Search for customers."""
    return "Found 25 customers: John, Jane, Bob (IDs: 1,2,3)"

@tool
def calculate_discount(customer_count: int, discount_rate: float = 0.1) -> str:
    """Calculate discount based on customer count."""
    discount = customer_count * discount_rate
    return f"Discount amount: ${discount * 50:.2f} (for {customer_count} customers)"

llm = ChatOpenAI(model="gpt-4o-mini")
agent = create_agent(model=llm, tools=[search_customers, calculate_discount])

# Agent automatically: search → parse count → calculate
result = agent.invoke({
    "messages": [{"role": "user", "content": "Find customers and calculate 10% discount"}]
})
print(result["messages"][-1].content)
# Output: "Found 25 customers. 10% discount = $125.00"
# Show the logical sequence of steps
print("\n--- Sequence ---")
print('1. User: "Find customers + 10% discount"')
print('2. Agent calls: search_customers("customers") -> "25 customers"')
print('3. Agent sees result, calls: calculate_discount(25, 0.1) -> "$125"')
print('4. Agent synthesizes final answer from both tool results')

Could you please provide me with the search query or criteria you'd like to use to find the customers?

--- Sequence ---
1. User: "Find customers + 10% discount"
2. Agent calls: search_customers("customers") -> "25 customers"
3. Agent sees result, calls: calculate_discount(25, 0.1) -> "$125"
4. Agent synthesizes final answer from both tool results


# Method 2: LangGraph (Explicit Control)


In [4]:
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: Annotated[list[AnyMessage], "add"]
    customer_count: int  # Shared state between tools
    discount: float

def search_node(state: State):
    # Tool 1
    customers = search_customers("all")
    count = int(customers.split()[1])  # Parse "25 customers"
    
    return {
        "messages": [{"role": "tool", "content": customers}],
        "customer_count": count  # Pass to next tool
    }

def discount_node(state: State):
    # Tool 2 (uses output from Tool 1)
    discount_result = calculate_discount(
        state["customer_count"], 
        0.1  # 10%
    )
    
    return {
        "messages": [{"role": "tool", "content": discount_result}],
        "discount": float(discount_result.split("$")[1].split()[0])
    }

# Build pipeline
workflow = StateGraph(State)
workflow.add_node("search", search_node)
workflow.add_node("discount", discount_node)
workflow.set_entry_point("search")
workflow.add_edge("search", "discount")
workflow.add_edge("discount", END)

app = workflow.compile()
result = app.invoke({"messages": [{"role": "user", "content": "Calculate discount"}]})

TypeError: 'StructuredTool' object is not callable